In [1]:
# SECTION 1: Setup and Imports
!pip install fpdf
!pip install scikit-learn
!pip install seaborn
!pip install yellowbrick

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=c4ea4d9b6c9d83f1f6f9ae97a4bb68eff49c6034c6b78d59d4ab52818e6203a7
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
from fpdf import FPDF
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score, silhouette_score
from sklearn.decomposition import PCA
from google.colab import files
warnings.filterwarnings('ignore')

# Plot settings for Colab
plt.style.use('default')
sns.set_theme()
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['font.size'] = 12
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12

# Upload files
from google.colab import files
uploaded = files.upload()  # This will prompt you to upload your CSV files

Saving Customers.csv to Customers.csv
Saving Products.csv to Products.csv
Saving Transactions.csv to Transactions.csv


In [5]:
"""
TASK 1: EXPLORATORY DATA ANALYSIS (EDA)
- Data preprocessing
- Basic data visualization
- Understanding data patterns
"""

def preprocess_data(customers_df, products_df, transactions_df):
    """Handle missing values and preprocess the data"""
    # Fill missing values in transactions
    transactions_df['Quantity'] = transactions_df['Quantity'].fillna(0)
    transactions_df['TotalValue'] = transactions_df['TotalValue'].fillna(0)
    transactions_df['Price'] = transactions_df['Price'].fillna(transactions_df['Price'].mean())

    # Fill missing values in products
    products_df['Price'] = products_df['Price'].fillna(products_df['Price'].mean())
    products_df['Category'] = products_df['Category'].fillna('Other')

    # Fill missing values in customers
    customers_df['Region'] = customers_df['Region'].fillna('Unknown')

    return customers_df, products_df, transactions_df

def perform_eda(customers_df, products_df, transactions_df):
    """Perform Exploratory Data Analysis"""
    # Create EDA visualizations
    fig = plt.figure(figsize=(15, 12))
    gs = fig.add_gridspec(2, 2, hspace=0.4, wspace=0.3)

    # 1. Regional Distribution
    ax1 = fig.add_subplot(gs[0, 0])
    region_counts = customers_df['Region'].value_counts()
    sns.barplot(x=region_counts.index, y=region_counts.values, palette='viridis', ax=ax1)
    ax1.set_title('Customer Distribution by Region', pad=15)
    ax1.set_xlabel('Region')
    ax1.set_ylabel('Number of Customers')
    ax1.tick_params(axis='x', rotation=45)

    # 2. Product Categories
    ax2 = fig.add_subplot(gs[0, 1])
    category_counts = products_df['Category'].value_counts()
    sns.barplot(x=category_counts.index, y=category_counts.values, palette='viridis', ax=ax2)
    ax2.set_title('Product Distribution by Category', pad=15)
    ax2.set_xlabel('Category')
    ax2.set_ylabel('Number of Products')
    ax2.tick_params(axis='x', rotation=45)

    # 3. Monthly Sales Trend
    ax3 = fig.add_subplot(gs[1, 0])
    monthly_sales = transactions_df.groupby(pd.Grouper(key='TransactionDate', freq='M'))['TotalValue'].sum()
    ax3.plot(range(len(monthly_sales)), monthly_sales.values, marker='o', linewidth=2, color='#2ecc71')
    ax3.set_title('Monthly Sales Trend', pad=15)
    ax3.set_xlabel('Month')
    ax3.set_ylabel('Total Sales ($)')
    ax3.tick_params(axis='x', rotation=45)
    ax3.grid(True, linestyle='--', alpha=0.7)

    # 4. Price Distribution
    ax4 = fig.add_subplot(gs[1, 1])
    sns.histplot(data=products_df, x='Price', bins=30, color='#3498db', ax=ax4)
    ax4.set_title('Product Price Distribution', pad=15)
    ax4.set_xlabel('Price ($)')
    ax4.set_ylabel('Count')
    ax4.grid(True, linestyle='--', alpha=0.7)

    plt.tight_layout()
    plt.savefig('Swetha_Magesh_EDA_Plots.png', dpi=300, bbox_inches='tight')
    plt.close()

"""
TASK 2: BUSINESS INSIGHTS
- Calculate key metrics
- Generate business insights
- Create PDF report
"""

def generate_business_insights(customers_df, products_df, transactions_df):
    """Generate business insights"""
    # Calculate metrics
    customer_metrics = transactions_df.groupby('CustomerID').agg({
        'TransactionID': 'count',
        'TotalValue': ['sum', 'mean'],
        'Quantity': 'sum'
    }).round(2)

    regional_performance = transactions_df.merge(
        customers_df[['CustomerID', 'Region']]
    ).groupby('Region').agg({
        'TotalValue': ['sum', 'mean'],
        'TransactionID': 'count'
    }).round(2)

    regional_performance.columns = ['Total Sales ($)', 'Avg Transaction ($)', 'Number of Transactions']

    insights = f"""
    Business Insights Report
    {'='*50}

    1. Customer Analysis
    {'-'*30}
    - Total Customers: {len(customers_df):,}
    - Average Transactions per Customer: {customer_metrics['TransactionID']['count'].mean():.2f}
    - Average Customer Spend: ${customer_metrics['TotalValue']['sum'].mean():,.2f}

    2. Product Analysis
    {'-'*30}
    - Total Products: {len(products_df):,}
    - Average Price: ${products_df['Price'].mean():.2f}
    - Price Range: ${products_df['Price'].min():.2f} - ${products_df['Price'].max():.2f}
    - Top Categories by Revenue:
    {transactions_df.merge(products_df[['ProductID', 'Category']], on='ProductID')
    .groupby('Category')['TotalValue'].sum()
    .sort_values(ascending=False)
    .head(3)
    .to_string()}

    3. Sales Analysis
    {'-'*30}
    - Total Revenue: ${transactions_df['TotalValue'].sum():,.2f}
    - Average Transaction Value: ${transactions_df['TotalValue'].mean():.2f}
    - Most Active Region: {regional_performance['Number of Transactions'].idxmax()}

    4. Regional Performance
    {'-'*30}
    {regional_performance.to_string()}
    """

    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.multi_cell(0, 10, insights)
    pdf.output("Swetha_Magesh_Business_Insights.pdf")

"""
TASK 3: LOOKALIKE MODEL
- Feature engineering (RFM analysis)
- Customer similarity calculation
- Generate recommendations
"""

def create_customer_features(customers_df, transactions_df, products_df):
    """Create customer features for modeling"""
    # RFM Analysis
    latest_date = transactions_df['TransactionDate'].max()

    rfm = transactions_df.groupby('CustomerID').agg({
        'TransactionDate': lambda x: (latest_date - x.max()).days,
        'TransactionID': 'count',
        'TotalValue': ['sum', 'mean', 'std'],
        'Quantity': ['sum', 'mean']
    }).fillna(0)

    rfm.columns = ['recency', 'frequency', 'total_spend', 'avg_transaction',
                  'std_transaction', 'total_items', 'avg_items']

    # Category preferences
    trans_prod = transactions_df.merge(products_df[['ProductID', 'Category']], on='ProductID')
    category_pivot = pd.crosstab(
        trans_prod['CustomerID'],
        trans_prod['Category'],
        values=trans_prod['Quantity'],
        aggfunc='sum',
        normalize='index'
    ).fillna(0)

    # Combine features
    customer_features = rfm.join(category_pivot, how='left').fillna(0)
    return customer_features

def generate_lookalikes(customers_df, feature_matrix, n_recommendations=3):
    """Generate lookalike customers"""
    scaler = RobustScaler()
    normalized_features = scaler.fit_transform(feature_matrix)
    similarity_matrix = cosine_similarity(normalized_features)

    results = {}
    first_20_customers = customers_df['CustomerID'].iloc[:20]

    for cust_id in first_20_customers:
        idx = feature_matrix.index.get_loc(cust_id)
        similarities = similarity_matrix[idx]
        similar_indices = np.argsort(similarities)[::-1][1:n_recommendations+1]

        results[cust_id] = [
            (feature_matrix.index[i], round(similarities[i], 4))
            for i in similar_indices
        ]

    lookalike_df = pd.DataFrame([
        (k, v[0][0], v[0][1], v[1][0], v[1][1], v[2][0], v[2][1])
        for k, v in results.items()
    ], columns=['CustomerID', 'Lookalike1', 'Score1',
                'Lookalike2', 'Score2', 'Lookalike3', 'Score3'])

    lookalike_df.to_csv('Swetha_Magesh_Lookalike.csv', index=False)
    return lookalike_df

"""
TASK 4: CUSTOMER SEGMENTATION
- Determine optimal number of clusters
- Perform K-means clustering
- Generate cluster characteristics
- Visualize results
"""

def perform_clustering(feature_matrix):
    """Perform customer segmentation"""
    # Remove any remaining NaN values
    feature_matrix = feature_matrix.fillna(0)

    # Scale features
    scaler = RobustScaler()
    X = scaler.fit_transform(feature_matrix)

    # Find optimal clusters
    scores = []
    for k in range(2, 11):
        kmeans = KMeans(n_clusters=k, random_state=42)
        labels = kmeans.fit_predict(X)
        scores.append({
            'k': k,
            'db_score': davies_bouldin_score(X, labels),
            'silhouette': silhouette_score(X, labels)
        })

    scores_df = pd.DataFrame(scores)
    optimal_k = scores_df.loc[scores_df['db_score'].idxmin(), 'k']

    # Final clustering
    final_kmeans = KMeans(n_clusters=int(optimal_k), random_state=42)
    labels = final_kmeans.fit_predict(X)

    # Calculate characteristics
    cluster_profiles = pd.DataFrame(X, columns=feature_matrix.columns, index=feature_matrix.index)
    cluster_profiles['Cluster'] = labels

    characteristics = cluster_profiles.groupby('Cluster').mean().round(3)

    formatted_characteristics = pd.DataFrame({
        'Size': pd.Series(labels).value_counts().sort_index(),
        'Avg Transactions': characteristics['frequency'],
        'Avg Spend ($)': characteristics['total_spend'],
        'Recency (days)': characteristics['recency'],
        'Books (%)': characteristics['Books'] * 100 if 'Books' in characteristics else 0,
        'Clothing (%)': characteristics['Clothing'] * 100 if 'Clothing' in characteristics else 0,
        'Electronics (%)': characteristics['Electronics'] * 100 if 'Electronics' in characteristics else 0,
        'Home Decor (%)': characteristics['Home Decor'] * 100 if 'Home Decor' in characteristics else 0
    }).round(2)

    # Visualize clusters
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X)

    plt.figure(figsize=(10, 8))
    scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1],
                         c=labels, cmap='viridis',
                         s=100, alpha=0.6)
    plt.title('Customer Segments Visualization', pad=20)
    plt.xlabel('First Principal Component')
    plt.ylabel('Second Principal Component')
    cbar = plt.colorbar(scatter)
    cbar.set_label('Cluster')
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.savefig('Swetha_Magesh_Clusters.png', dpi=300, bbox_inches='tight')
    plt.close()

    clustering_report = f"""
    Customer Segmentation Results
    {'='*50}

    Key Metrics:
    - Number of Clusters: {int(optimal_k)}
    - Davies-Bouldin Index: {scores_df.loc[scores_df['k'] == optimal_k, 'db_score'].iloc[0]:.4f}
    - Silhouette Score: {scores_df.loc[scores_df['k'] == optimal_k, 'silhouette'].iloc[0]:.4f}

    Cluster Characteristics:
    {formatted_characteristics.to_string()}
    """

    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.multi_cell(0, 10, clustering_report)
    pdf.output("Swetha_Magesh_Clustering.pdf")

    return labels, clustering_report

"""
MAIN EXECUTION
Execute all tasks in sequence
"""

def main():
    try:
        # Load and preprocess data
        print("Loading and preprocessing data...")
        customers_df = pd.read_csv('Customers.csv')
        products_df = pd.read_csv('Products.csv')
        transactions_df = pd.read_csv('Transactions.csv')

        customers_df, products_df, transactions_df = preprocess_data(
            customers_df, products_df, transactions_df
        )

        # Convert dates
        customers_df['SignupDate'] = pd.to_datetime(customers_df['SignupDate'])
        transactions_df['TransactionDate'] = pd.to_datetime(transactions_df['TransactionDate'])

        # Execute tasks
        print("Performing EDA...")
        perform_eda(customers_df, products_df, transactions_df)

        print("Generating Business Insights...")
        generate_business_insights(customers_df, products_df, transactions_df)

        print("Creating Customer Features...")
        feature_matrix = create_customer_features(customers_df, transactions_df, products_df)

        print("Generating Lookalikes...")
        lookalike_results = generate_lookalikes(customers_df, feature_matrix)

        print("Performing Clustering...")
        labels, clustering_report = perform_clustering(feature_matrix)

        # Download files
        print("\nDownloading generated files...")
        files.download('Swetha_Magesh_EDA_Plots.png')
        files.download('Swetha_Magesh_Business_Insights.pdf')
        files.download('Swetha_Magesh_Lookalike.csv')
        files.download('Swetha_Magesh_Clusters.png')
        files.download('Swetha_Magesh_Clustering.pdf')

        print("\nAnalysis complete! All files have been generated and downloaded.")

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        raise

if __name__ == "__main__":
    main()

Loading and preprocessing data...
Performing EDA...
Generating Business Insights...
Creating Customer Features...
Generating Lookalikes...
Performing Clustering...



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Analysis complete! All files have been generated and downloaded.
